# 01 加载数据文件
为了解决大模型数据时效性和幻觉两大问题，检索增强生成（RAG，Retrieval Augmented Generation）应运而生。RAG为大语言模型 (LLMs) 提供了从数据源检索信息的能力，并以此为基础生成回答。
![image.png](imgs/RAG.png)
借用[langchain chatchat](https://github.com/chatchat-space/Langchain-Chatchat?tab=readme-ov-file)项目的原理图，过程包括加载文件 -> 读取文本 -> 文本分割 -> 文本向量化 -> 问句向量化 -> 在文本向量中匹配出与问句向量最相似的 top k个 -> 匹配出的文本作为上下文和问题一起添加到 prompt中 -> 提交给 LLM生成回答。本文将按照上图序号依次给出实现代码，最终拼凑成一个简单的RAG应用。

上图的步骤1-7是对数据源的处理，包含了大量的dirty work（至少我是这样认为的）。并且数据源处理对于RAG的效果具有决定性的影响，所以这部分工作往往会需要大量时间和注意力。

本节对应上图第1、2个步骤，加载文档，并将文档中的信息提取为文本。
下面实现以下格式的加载
* txt
* Markdown
* CSV
* HTML
* JSON
* PDF
* python脚本

In [7]:
# txt
from langchain.document_loaders import TextLoader

loader_txt = TextLoader('data/index.txt')
txt = loader_txt.load()

print(len(txt))
print(txt[0].page_content)  # 打印txt文本
print(txt[0].metadata['source'])  # 打印txt的地址


1
荷塘月色

作者: 朱自清

　
　　这几天心里颇不宁静。今晚在院子里坐着乘凉，忽然想起日日走过的荷塘，在这满月的光里，总该另有一番样子吧。月亮渐渐地升高了，墙外马路上孩子们的欢笑，已经听不见了；妻在屋里拍着闰儿，迷迷糊糊地哼着眠歌。我悄悄地披了大衫，带上门出去。
　　沿着荷塘，是一条曲折的小煤屑路。这是一条幽僻的路；白天也少人走，夜晚更加寂寞。荷塘四面，长着许多树，蓊蓊郁郁的。路的一旁，是些杨柳，和一些不知道名字的树。没有月光的晚上，这路上阴森森的，有些怕人。今晚却很好，虽然月光也还是淡淡的。
　　路上只我一个人，背着手踱着。这一片天地好像是我的；我也像超出了平常的自己，到了另一世界里。我爱热闹，也爱冷静；爱群居，也爱独处。像今晚上，一个人在这苍茫的月下，什么都可以想，什么都可以不想，便觉是个自由的人。白天里一定要做的事，一定要说的话，现在都可不理。这是独处的妙处，我且受用这无边的荷香月色好了。曲曲折折的荷塘上面，弥望的是田田的叶子。叶子出水很高，像亭亭的舞女的裙。层层的叶子中间，零星地点缀着些白花，有袅娜地开着的，有羞涩地打着朵儿的；正如一粒粒的明珠，又如碧天里的星星，又如刚出浴的美人。微风过处，送来缕缕清香，仿佛远处高楼上渺茫的歌声似的。这时候叶子与花也有一丝的颤动，像闪电般，霎时传过荷塘的那边去了。叶子本是肩并肩密密地挨着，这便宛然有了一道凝碧的波痕。叶子底下是脉脉的流水，遮住了，不能见一些颜色；而叶子却更见风致了。
月光如流水一般，静静地泻在这一片叶子和花上。薄薄的青雾浮起在荷塘里。叶子和花仿佛在牛乳中洗过一样；又像笼着轻纱的梦。虽然是满月，天上却有一层淡淡的云，所以不能朗照；但我以为这恰是到了好处——酣眠固不可少，小睡也别有风味的。月光是隔了树照过来的，高处丛生的灌木，落下参差的斑驳的黑影，峭楞楞如鬼一般；弯弯的杨柳的稀疏的倩影，却又像是画在荷叶上。塘中的月色并不均匀；但光与影有着和谐的旋律，如梵婀玲上奏着的名曲。
　　荷塘的四面，远远近近，高高低低都是树，而杨柳最多。这些树将一片荷塘重重围住；只在小路一旁，漏着几段空隙，像是特为月光留下的。树色一例是阴阴的，乍看像一团烟雾；但杨柳的丰姿，便在烟雾里也辨得出。树梢上隐隐约约的是一带远山，只有些大意罢了。树缝里也漏着一两点路灯光，没精打采的，是渴睡人的眼。这时候最热闹的，要数树上的蝉声与水里的蛙声；但热

In [9]:
# md
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

loader_md = UnstructuredMarkdownLoader(
    "data\langchain_README.md", 
    mode="elements"  # 可选的，如果设置mode="elements"，则会保持md文本块的分隔。
    )
md = loader_md.load()

print(len(md))  # 分隔了65个不同的文本块

# 打印前3个文本块
for doc in md[1:3]:
    print(doc)

print(md[0].metadata["category"])  # 第一个文本块是标题
print(md[0].page_content)  # 标题的内容
print(md[1].metadata["category"])  # 第二个文本块是文本


65
page_content='⚡ Build context-aware reasoning applications ⚡' metadata={'source': 'data\\langchain_README.md', 'last_modified': '2024-07-02T17:41:35', 'languages': ['eng'], 'parent_id': '75bf9e673608f94f443e4f8903039b64', 'filetype': 'text/markdown', 'file_directory': 'data', 'filename': 'langchain_README.md', 'category': 'NarrativeText'}
page_content='To help you ship LangChain apps to production faster, check out LangSmith. 
LangSmith is a unified developer platform for building, testing, and monitoring LLM applications. 
Fill out this form to speak with our sales team.' metadata={'source': 'data\\langchain_README.md', 'link_texts': ['LangSmith', 'LangSmith', 'this form'], 'link_urls': ['https://smith.langchain.com', 'https://smith.langchain.com', 'https://www.langchain.com/contact-sales'], 'link_start_indexes': [64, 76, 184], 'last_modified': '2024-07-02T17:41:35', 'languages': ['eng'], 'parent_id': '75bf9e673608f94f443e4f8903039b64', 'filetype': 'text/markdown', 'file_directory'

In [10]:
# csv
from langchain_community.document_loaders.csv_loader import CSVLoader

file_path = ("data\index.csv")

loader = CSVLoader(file_path=file_path)
data = loader.load()

for record in data[:2]:
    print(record)

# 将其中一行转为字典格式
content = data[1].page_content[:-3]
lines = content.strip().split('\n')
data_dict = {}
for line in lines:
    if ':' in line:
        key, value = line.split(':', 1)
        data_dict[key.strip()] = value.strip()

print(f"CSV文件 {data[1].metadata['source']} 的第 {data[1].metadata['row']} 行，数据为{data_dict}" )

page_content='name: Zion
id: 001
gender: male
: ' metadata={'source': 'data\\index.csv', 'row': 0}
page_content='name: Lily
id: 002
gender: female
: ' metadata={'source': 'data\\index.csv', 'row': 1}
CSV文件 data\index.csv 的第 1 行，数据为{'name': 'Lily', 'id': '002', 'gender': 'female'}


In [11]:
# HTML
from langchain_community.document_loaders import BSHTMLLoader

loader = BSHTMLLoader("data\index.html")
data = loader.load()

print(data)
print(data[0].metadata["title"])

[Document(metadata={'source': 'data\\index.html', 'title': '示例HTML文件'}, page_content='\n\n\n\n示例HTML文件\n\n\n\n欢迎来到我的网站\n\n\n\n关于本站\n这是一个简单的HTML文件示例，旨在展示基本的HTML结构。\n\n\n联系我们\n如果你有任何问题或建议，请通过以下链接联系我们：\n发送邮件\n\n\n\n版权所有 © 2024\n\n\n\n')]
示例HTML文件


In [12]:
# JSON，仅适用于linux。
# windows不可用
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path='data/facebook_chat.json',
    jq_schema='.messages[].content',
    text_content=False)

data = loader.load()

ImportError: jq package not found, please install it with `pip install jq`

In [13]:
# PDF
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "data/2103.15348v2_2.pdf"
)
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

pages[0]

Document(metadata={'source': 'data/2103.15348v2_2.pdf', 'page': 0}, page_content='LayoutParser : A Uniﬁed Toolkit for Deep\nLearning Based Document Image Analysis\nZejiang Shen1( \x00), Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain\nLee4, Jacob Carlson3, and Weining Li5\n1Allen Institute for AI\nshannons@allenai.org\n2Brown University\nruochen zhang@brown.edu\n3Harvard University\n{melissadell,jacob carlson }@fas.harvard.edu\n4University of Washington\nbcgl@cs.washington.edu\n5University of Waterloo\nw422li@uwaterloo.ca\nAbstract. Recent advances in document image analysis (DIA) have been\nprimarily driven by the application of neural networks. Ideally, research\noutcomes could be easily deployed in production and extended for further\ninvestigation. However, various factors like loosely organized codebases\nand sophisticated model conﬁgurations complicate the easy reuse of im-\nportant innovations by a wide audience. Though there have been on-going\neﬀorts to improve reusabi

很多PDF是扫描文档，只提取文字是不够的。使用`rapidocr-onnxruntime`可以将图像提取为文字。
```shell
pip install --upgrade --quiet rapidocr-onnxruntime

In [15]:
# PDF with images
loader = PyPDFLoader("data/2103.15348v2_2.pdf", extract_images=True)
pages = loader.load()
print(len(pages))
pages[3].page_content

16


'4 Z. Shen et al.\nEfficient Data AnnotationC u s t o m i z e d  M o d e l  T r a i n i n gModel Cust omizationDI A Model HubDI A Pipeline SharingCommunity PlatformLa y out Detection ModelsDocument Images \nT h e  C o r e  L a y o u t P a r s e r  L i b r a r yOCR ModuleSt or age & VisualizationLa y out Data Structur e\nFig. 1: The overall architecture of LayoutParser . For an input document image,\nthe core LayoutParser library provides a set of oﬀ-the-shelf tools for layout\ndetection, OCR, visualization, and storage, backed by a carefully designed layout\ndata structure. LayoutParser also supports high level customization via eﬃcient\nlayout annotation and model training functions. These improve model accuracy\non the target samples. The community platform enables the easy sharing of DIA\nmodels and whole digitization pipelines to promote reusability and reproducibility.\nA collection of detailed documentation, tutorials and exemplar projects make\nLayoutParser easy to learn and use

In [16]:
# python脚本
from langchain.document_loaders import PythonLoader
loader = PythonLoader('data\hello.py')
docs = loader.load()
docs

[Document(metadata={'source': 'data\\hello.py'}, page_content='print("hello,LLMs")')]

熟悉了以上加载文档的类，下面可以使用更通用的方法。

In [17]:
# 批量的加载选定文件夹内指定格式的文档
from langchain_community.document_loaders import DirectoryLoader

file_path="data"
loader = DirectoryLoader(
    file_path,  # 选定的文件夹
    glob='**/*.txt',  # 加载所有txt
    show_progress=True,  # 显示进度条，可选
    use_multithreading=True  # 多线程加载，可选
    )

docs = loader.load()
print(len(docs))
print(docs)

100%|██████████| 2/2 [00:00<00:00, 34.11it/s]

2
[Document(metadata={'source': 'data\\index2.txt'}, page_content='这是另一个测试文件。'), Document(metadata={'source': 'data\\index.txt'}, page_content='荷塘月色\n\n作者: 朱自清\n\n这几天心里颇不宁静。今晚在院子里坐着乘凉，忽然想起日日走过的荷塘，在这满月的光里，总该另有一番样子吧。月亮渐渐地升高了，墙外马路上孩子们的欢笑，已经听不见了；妻在屋里拍着闰儿，迷迷糊糊地哼着眠歌。我悄悄地披了大衫，带上门出去。\n\n沿着荷塘，是一条曲折的小煤屑路。这是一条幽僻的路；白天也少人走，夜晚更加寂寞。荷塘四面，长着许多树，蓊蓊郁郁的。路的一旁，是些杨柳，和一些不知道名字的树。没有月光的晚上，这路上阴森森的，有些怕人。今晚却很好，虽然月光也还是淡淡的。\n\n路上只我一个人，背着手踱着。这一片天地好像是我的；我也像超出了平常的自己，到了另一世界里。我爱热闹，也爱冷静；爱群居，也爱独处。像今晚上，一个人在这苍茫的月下，什么都可以想，什么都可以不想，便觉是个自由的人。白天里一定要做的事，一定要说的话，现在都可不理。这是独处的妙处，我且受用这无边的荷香月色好了。曲曲折折的荷塘上面，弥望的是田田的叶子。叶子出水很高，像亭亭的舞女的裙。层层的叶子中间，零星地点缀着些白花，有袅娜地开着的，有羞涩地打着朵儿的；正如一粒粒的明珠，又如碧天里的星星，又如刚出浴的美人。微风过处，送来缕缕清香，仿佛远处高楼上渺茫的歌声似的。这时候叶子与花也有一丝的颤动，像闪电般，霎时传过荷塘的那边去了。叶子本是肩并肩密密地挨着，这便宛然有了一道凝碧的波痕。叶子底下是脉脉的流水，遮住了，不能见一些颜色；而叶子却更见风致了。\n\n月光如流水一般，静静地泻在这一片叶子和花上。薄薄的青雾浮起在荷塘里。叶子和花仿佛在牛乳中洗过一样；又像笼着轻纱的梦。虽然是满月，天上却有一层淡淡的云，所以不能朗照；但我以为这恰是到了好处——酣眠固不可少，小睡也别有风味的。月光是隔了树照过来的，高处丛生的灌木，落下参差的斑驳的黑影，峭楞楞如鬼一般；弯弯的杨柳的稀疏的倩影，却又像是画在荷叶上。塘中的月色并不均匀；但光与影有着和谐的旋律，如梵婀玲上奏着的名曲。\n\n荷塘的四面，远远近近，高高低低都是树，而

仍然使用`DirectoryLoader`，可以通过传入其他加载类的方式，加载其他格式的文件。下面以加载PDF为例

In [18]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader

file_path="data"
loader = DirectoryLoader(
    file_path,
    glob='**/*.PDF',
    loader_cls=PyPDFLoader  # 这里传入了加载PDF的类
    )

docs = loader.load()
print(len(docs))
print(docs)

16
[Document(metadata={'source': 'data\\2103.15348v2_2.pdf', 'page': 0}, page_content='LayoutParser : A Uniﬁed Toolkit for Deep\nLearning Based Document Image Analysis\nZejiang Shen1( \x00), Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain\nLee4, Jacob Carlson3, and Weining Li5\n1Allen Institute for AI\nshannons@allenai.org\n2Brown University\nruochen zhang@brown.edu\n3Harvard University\n{melissadell,jacob carlson }@fas.harvard.edu\n4University of Washington\nbcgl@cs.washington.edu\n5University of Waterloo\nw422li@uwaterloo.ca\nAbstract. Recent advances in document image analysis (DIA) have been\nprimarily driven by the application of neural networks. Ideally, research\noutcomes could be easily deployed in production and extended for further\ninvestigation. However, various factors like loosely organized codebases\nand sophisticated model conﬁgurations complicate the easy reuse of im-\nportant innovations by a wide audience. Though there have been on-going\neﬀorts to improve re